In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
%matplotlib inline

module_path = os.path.abspath(os.path.join("../lib"))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from shrinkage_estimators import covariance_linear_shrinkage
from shrinkage_estimators import precision_linear_shrinkage
from shrinkage_estimators import NERCOME

In [2]:
data_path = "../output/BOSS_DR12_NGC_z1/"
p = 18
T = np.loadtxt(os.path.join(data_path, f"T_{p}_{p}.matrix"))

In [3]:
def estimate_shrinkage_precision(P, target, label, n, v):
    psi, alpha, beta = precision_linear_shrinkage.linear_shrinkage(P, target)
    np.savetxt(
        os.path.join(data_path, f"matrices/n{n}/pre_shrinkage_{label}", f"pre_{p}_{p}_shrinkage_{label}_{n}_v{v+1}.matrix"),
        psi,
        header=f"alpha={alpha}, beta={beta}"
    )
    
    return alpha, beta

In [ ]:
def estimate_matrices(n):
    save_path = os.path.join(data_path, f"matrices/n{n}/")
    mocks_path = os.path.join(data_path, f"mocks/n{n}/")
    r = len([f for f in os.listdir(mocks_path) if not f.startswith('.')])
    
    # This is where we are going to save the shrinkage parameters during the run
    lambdas_Sii = np.empty(r)
    lambdas_P = np.empty(r)
    alphas_inv_then_diag = np.empty(r)
    betas_inv_then_diag = np.empty(r)
    alphas_diag_then_inv = np.empty(r)
    betas_diag_then_inv = np.empty(r)
    alphas_P = np.empty(r)
    betas_P = np.empty(r)
    ss_min = np.empty(r)

    for v in range(r):
        print(f"Run {v+1}")
        P = np.loadtxt(os.path.join(mocks_path, f"P_{p}_{n}_v{v+1}.matrix"))
    
        # Sample cov estimate
        S = np.cov(P)
        #np.savetxt(os.path.join(save_path, "cov_sample", f"cov_{p}_{p}_sample_{n}_v{v+1}.matrix"), S)
    
        # Shrinkage cov estimates
        C_shrinkage_Sii, lmbda_est_Sii = covariance_linear_shrinkage.linear_shrinkage(P)
        lambdas_Sii[v] = lmbda_est_Sii
        np.savetxt(
            os.path.join(save_path, "cov_shrinkage_Sii", f"cov_{p}_{p}_shrinkage_Sii_{n}_v{v+1}.matrix"), 
            C_shrinkage_Sii,
            header=f"lambda={lmbda_est_Sii}"
        )
    
        C_shrinkage_P, lmbda_est_P = covariance_linear_shrinkage.linear_shrinkage(P, target=T)
        lambdas_P[v] = lmbda_est_P
        np.savetxt(
            os.path.join(save_path, "cov_shrinkage_P", f"cov_{p}_{p}_shrinkage_P_{n}_v{v+1}.matrix"),
            C_shrinkage_P,
            header=f"lambda={lmbda_est_P}"
        )
    
        # Shrinkage precision matrix estimates
        alphas_diag_then_inv[v], betas_diag_then_inv[v] = estimate_shrinkage_precision(
            P,
            np.diag(1/np.diag(S)),
            "diag_then_inv",
            n, 
            v
        )
        
        alphas_inv_then_diag[v], betas_inv_then_diag[v] = estimate_shrinkage_precision(
            P,
            np.diag(np.diag(np.linalg.inv(S))),
            "inv_then_diag",
            n, 
            v
        )
        
        alphas_P[v], betas_P[v] = estimate_shrinkage_precision(
            P,
            np.linalg.inv(T),
            "P",
            n, 
            v
        )
        
        # NERCOME cov estimate
        C_nercome, s_min = NERCOME.NERCOME(P, all_splits=(n <= 50)) # Only do all splits if n at most 50
        ss_min[v] = s_min
        np.savetxt(
            os.path.join(save_path, "cov_NERCOME", f"cov_{p}_{p}_NERCOME_{n}_v{v+1}.matrix"),
            C_nercome,
            header=f"s_min={s_min}"
        )
    
    np.savetxt(os.path.join(save_path, f"lambdas_Sii_{n}.dat"), lambdas_Sii)
    np.savetxt(os.path.join(save_path, f"lambdas_P_{n}.dat"), lambdas_P)
    np.savetxt(os.path.join(save_path, f"alphas_inv_then_diag_{n}.dat"), alphas_inv_then_diag)
    np.savetxt(os.path.join(save_path, f"betas_inv_then_diag_{n}.dat"), betas_inv_then_diag)
    np.savetxt(os.path.join(save_path, f"alphas_diag_then_inv_{n}.dat"), alphas_diag_then_inv)
    np.savetxt(os.path.join(save_path, f"betas_diag_then_inv_{n}.dat"), betas_diag_then_inv)
    np.savetxt(os.path.join(save_path, f"alphas_P_{n}.dat"), alphas_P)
    np.savetxt(os.path.join(save_path, f"betas_P_{n}.dat"), betas_P)
    np.savetxt(os.path.join(save_path, f"ss_min_{n}.dat"), ss_min)

In [ ]:
estimate_matrices(21)
estimate_matrices(30)
estimate_matrices(2048)